In [ ]:
import numpy as np
import math as m
import matplotlib.pyplot as plt
from sklearn . datasets import load_digits
import timeit
from prettytable import PrettyTable

# Exercise 2 Part 1

First passing the code for newton's method using regularisation

In [ ]:
#first we will define the function, its gradient and the hessian

def func(A,Y,x,l):
    f = 0.5 * (np.linalg.norm(A @ x - y))**2 + 0.5*l* x.T @ x
    return f

def grad(A,Y,x, l):

    #for the OLS the gradient is equal to
    # A^T (Ax - Y)

    g = A.T @ A @ x - A.T @ Y + l*x

    return g

def hess(A,Y,x,l):

    #we define hessian as the double derivative of the function f
    #hence we simple take the derivative of the gradient
    #anyways the dimensions are independent of each other

    h = A.T @ A + l*np.identity(A.shape [1], dtype = float)

    return h


# Define tolerance level
def tolerance(p):
    return 10 ** (-p)

# Define the newton optimisation algorithm
def newton_opt_reg(x, A, y, l):

        history = []
        TLevel = tolerance(4) #tolerance of 10^-4
        count = 0

    # Perform the newton optimisation iterations

        while (np.linalg.norm(grad(A, y, x, l)) > TLevel):

            h = hess(A,y, x,l)
            hess_inv = np.linalg.inv(h)


            #initialise values for our a, gamma, and p
            #a = 0.99 (for inexact line search)
            #gamma = 0.5 (for inexact line search)
            #p = 0.5 (for inexact line search)

            d = -grad(A,y,x,l)

            #now a loop to conduct inexact line search
            #while (func(A,y,x + a*d,l ) > (func(A,y,x,l) + gamma*a*(grad(A,y,x,l).T@d))):
                #a = p*a


            # Update the parameters
            #x = x + a * hess_inv @ d (for inexact line search)
            x = x + hess_inv @ d

            # Save the history of the parameters
            history.append((x, func(A,y, x, l), count))

            #count the iterations for convergence
            count = count+1


            #print(hess_inv)
            #print(a)
            #print(count)

            #print(x)
            #print(np.linalg.norm(grad(A, y, x)))
            #print()


        return x

In [ ]:
np. random . seed (10) #for repeatability

N = 200

ds = [1000 , 5000 , 10000 , 20000 , 25000 , 50000 , 100000 , 200000 , 500000 , 1000000]

lambda_reg = 0.001

eps = np. random . randn (N ,1) # random noise

#also for tabulation purposes
T = [] #time taken
Diff = [] #L2 diff
Final_func = [] #final functional value

#for tabulation purposes
table = PrettyTable()
table.field_names = ['Dimension', 'Time Taken', 'Functional Value', 'L2 Difference']

#For each value of dimension in the ds array , we will check the behavior of
#Newton method
for i in range (np. size (ds)):
  d=ds[i]
  A = np. random . randn (N,d)
  # Normalize the columns
  for j in range (A. shape [1]) :
    A[:,j] = A[:,j]/ np. linalg . norm (A[:,j])


  xorig = np. ones ((d ,1))
  y = np.dot(A, xorig ) + eps
  start = timeit . default_timer ()

  #also passing initial value for x
  x0 = np.zeros([d ,1])

  # call Newton method with A,y, lambda and obtain the optimal solution x_opt
  x_opt = newton_opt_reg(x0, A,y, lambda_reg )

  newtontime = timeit . default_timer () - start # time is in seconds

  # print the total time and the L2 norm difference || x_opt - xorig || for Newton method
  print("For dimension = ", d)
  print("Time taken = ", newtontime)
  print("Final Functional Value = ", func(A,y,x_opt,lambda_reg))
  print("L2 difference = ", np.linalg.norm(x_opt - xorig))
  print()

  T.append(newtontime)
  Diff.append(np.linalg.norm(x_opt - xorig))
  Final_func.append(func(A,y,x_opt,lambda_reg))

  table.add_row([ds[i], T[i], Final_func[i], Diff[i]])
  print(table)
  print()


For dimension =  1000
Time taken =  0.20516580900000747
Final Functional Value =  [[0.14183477]]
L2 difference =  28.754555061518257

+-----------+---------------------+------------------+--------------------+
| Dimension |      Time Taken     | Functional Value |   L2 Difference    |
+-----------+---------------------+------------------+--------------------+
|    1000   | 0.20516580900000747 |  [[0.14183477]]  | 28.754555061518257 |
+-----------+---------------------+------------------+--------------------+

For dimension =  5000
Time taken =  17.37835525
Final Functional Value =  [[0.10576242]]
L2 difference =  69.32541692127397

+-----------+---------------------+------------------+--------------------+
| Dimension |      Time Taken     | Functional Value |   L2 Difference    |
+-----------+---------------------+------------------+--------------------+
|    1000   | 0.20516580900000747 |  [[0.14183477]]  | 28.754555061518257 |
|    5000   |     17.37835525     |  [[0.10576242]]  | 6

Session crashed in 3 iterations

Hence we will only tabulate for 3 iterations

# Exercise 2 Part 2

For BFGS method

In [ ]:
#first we will define the function, its gradient and the hessian

def func(A,Y,x,l):
    f = 0.5 * (np.linalg.norm(A @ x - y))**2 + 0.5*l* x.T @ x
    return f

def grad(A,Y,x, l):

    #for the OLS the gradient is equal to
    # A^T (Ax - Y)

    g = A.T @ A @ x - A.T @ Y + l*x

    return g

def hess(A,Y,x,l):

    #we define hessian as the double derivative of the function f
    #hence we simple take the derivative of the gradient
    #anyways the dimensions are independent of each other

    h = A.T @ A + l*np.identity(A.shape[1], dtype = float)

    return h



# Define tolerance level
def tolerance(p):
    return 10 ** (-p)


def L2_norm(g):
    return np.linalg.norm(g)


def BFGS_line_search_reg(x, A, y, B0,l):

    g = grad(A,y,x,l)

    history = []
    TLevel = tolerance(4) #setting tolerance to 4
    count = 0

    while (L2_norm(g) > TLevel):

        #initialising line search parameters

        a0 = 0.99
        p = 0.5
        gamma = 0.5

        #initialising the direction parameter

        d =  - B0 @ g

        #now the line search loop

        while (func(A,y,x + a0*d,l) > func(A,y,x,l) + gamma*a0*g.T@ d): # Check the RHS once, whether it is supposed to be g.T @ d or d.T @ d
            a0 = a0 * p


        #initialising storage matrices
        x_old = np.zeros((A.shape[1],1))
        x_new = np.zeros((A.shape[1],1))

        #storing old values of x_k
        x_old = x

        #computing x_(k+1)
        x = x + a0*d

        #storing x_(k+1)
        x_new = x


        #Intitialising BFGS parameters
        s = np.zeros((A.shape[1],1))
        z = np.zeros((A.shape[1],1))
        I = np.identity(A.shape[1])

        #giving values to these two their values
        s = x_new - x_old
        z = grad(A, y, x_new, l) - grad(A, y, x_old, l)

        #Updating B using s and y
        B0 = (I - (1/(s.T @ z))* s @ z.T) @ B0 @ (I - (1/(s.T @ z))* z @ s.T) + (1/(s.T @ z))* s @ s.T

        #now updating the gradient
        g = grad(A, y, x, l)

        # Save the history of the parameters
        history.append((x, func(A, y, x, l), count))

        #count the iterations for convergence
        count = count+1


    return x


In [ ]:
np. random . seed (10) #for repeatability

N = 200

ds = [1000 , 5000 , 10000 , 20000 , 25000 , 50000 , 100000 , 200000 , 500000 , 1000000]

lambda_reg = 0.001

eps = np. random . randn (N ,1) # random noise

#also for tabulation purposes
T_bfgs = [] #time taken
Diff_bfgs = [] #L2 diff
Final_func_bfgs = [] #final functional value

#making the table
table_bfgs = PrettyTable()
table_bfgs.field_names = ['Dimension', 'Time Taken', 'Functional Value', 'L2 Difference']

#For each value of dimension in the ds array , we will check the behavior of
#Newton method
for i in range (np. size (ds)):
  d=ds[i]
  A = np. random . randn (N,d)
  # Normalize the columns
  for j in range (A. shape [1]) :
    A[:,j] = A[:,j]/ np. linalg . norm (A[:,j])


  xorig = np. ones ((d ,1))
  y = np.dot(A, xorig ) + eps
  start = timeit . default_timer ()

  #also passing initial value for x
  x0 = np.zeros([d ,1])

  #taking b0 as identity matrix
  b0 = np.identity(d)

  # call BFGS method with A,y, lambda and obtain the optimal solution x_opt
  x_opt_bfgs = BFGS_line_search_reg(x0, A,y, b0, lambda_reg )

  bfgstime = timeit . default_timer () - start # time is in seconds

  # print the total time and the L2 norm difference || x_opt_bfgs - xorig || for BFGS method
  print("For dimension = ", d)
  print("Time taken = ", bfgstime)
  print("Final Functional Value = ", func(A,y,x_opt_bfgs,lambda_reg))
  print("L2 difference = ", np.linalg.norm(x_opt_bfgs - xorig))
  print()

  T_bfgs.append(bfgstime)
  Diff_bfgs.append(np.linalg.norm(x_opt_bfgs - xorig))
  Final_func_bfgs.append(func(A,y,x_opt_bfgs,lambda_reg))

  #tabulating
  table_bfgs.add_row([ds[i], T_bfgs[i], Final_func_bfgs[i], Diff_bfgs[i]])

  print(table_bfgs)
  print()


For dimension =  1000
Time taken =  4.0553912740000015
Final Functional Value =  [[0.14183477]]
L2 difference =  28.75455478489831

+-----------+--------------------+------------------+-------------------+
| Dimension |     Time Taken     | Functional Value |   L2 Difference   |
+-----------+--------------------+------------------+-------------------+
|    1000   | 4.0553912740000015 |  [[0.14183477]]  | 28.75455478489831 |
+-----------+--------------------+------------------+-------------------+

For dimension =  5000
Time taken =  418.35501031399997
Final Functional Value =  [[0.10576242]]
L2 difference =  69.32541690212179

+-----------+--------------------+------------------+-------------------+
| Dimension |     Time Taken     | Functional Value |   L2 Difference   |
+-----------+--------------------+------------------+-------------------+
|    1000   | 4.0553912740000015 |  [[0.14183477]]  | 28.75455478489831 |
|    5000   | 418.35501031399997 |  [[0.10576242]]  | 69.325416902121

KeyboardInterrupt: 

The above fails after 2nd iteration (third iteration takes 1 hour).